# Base firestore

* jump to relevant section after this

In [240]:
import pandas as pd
import firebase_admin
from IPython import display
from IPython.display import display, Markdown

from firebase_admin import firestore
from firebase_admin import credentials
class Firestore:
    def __init__(self,service_account_file="C:/i/auth/run-pix-092258e3cb1b.json"):
        cred = credentials.Certificate(service_account_file)
        try: self.app = firebase_admin.initialize_app(cred)
        except: self.app = firebase_admin.get_app()
        self.fstore=firestore.client()
    "delete all subnodes"
    def delete_all_docs(self,collection):
        for n in collection.get():
            print(n.__dict__)
            n._reference.delete()
    def collection(self):
        return self.fstore
    
    
runpix=Firestore()

In [3]:
#runpix.delete_all_docs(runpix.fstore.collection('races').document('test').collection('readings'))

# RAces

In [159]:

races=runpix.fstore.collection('races')
"list all races"
def _f(x): 
    _={"id":x.id,'ref':x}
    try: _.update(x.get().to_dict())
    except: pass
    return _
def fs2Df(collPath):
    return pd.DataFrame([_f(_) for _ in runpix.fstore.collection(collPath).list_documents()])
df_races=fs2Df('races')
df_races

,id,ref,Date,bibPatterm,Waypoints,Name,Location,bibPattern,photoStatus
0,default,<google.cloud.firestore_v1.document.DocumentRe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mychoice23apr,<google.cloud.firestore_v1.document.DocumentRe...,YYYY-MM-DD,\d\d\d\d,"[venue, start, end]",Default Race Year Month,DC,NaN,NaN
2,mychoice23feb,<google.cloud.firestore_v1.document.DocumentRe...,2023-02-11,NaN,"[venue, end]",My Choice 2023 February,GT,\d\d\d\d,[available]
3,mychoice23jan,<google.cloud.firestore_v1.document.DocumentRe...,2023-03-12,NaN,"[venue, start, end]",My Choice 2023 Januaryr,DC,NaN,NaN
4,mychoice23mar,<google.cloud.firestore_v1.document.DocumentRe...,2023-03-12,NaN,"[venue, start, end]",My Choice 2023 March,DC,NaN,NaN
5,werun2023,<google.cloud.firestore_v1.document.DocumentRe...,2023-03-05,NaN,"[venue, route]",We Run 2023 (4th Edition),Nigadi Pradhikaram,\d{4},[available]


## check all images in the race

In [160]:
raceId='werun2023'
df_images=fs2Df(f'races/{raceId}/images')
df_images.shape

(7, 7)

In [161]:
# df_images['imagePath'].value_counts()
def checkBlobs(x,types=['processed','thumbs']):
    return [bucket.blob(f'{typ}/{raceId}/{x}').exists() 
                                        for typ in types]
df_images['imagePath'].apply(lambda x: checkBlobs(x))

0    [True, True]
1    [True, True]
2    [True, True]
3    [True, True]
4    [True, True]
5    [True, True]
6    [True, True]
Name: imagePath, dtype: object

In [162]:
def deleteFSimages4missingBlobs(x,types=['processed','thumbs']):
    chk = checkBlobs(x.imagePath)
    if not any(chk):
        print(chk,x,x.imagePath)
        x.ref.delete()
df_images.apply(deleteFSimages4missingBlobs ,axis=1)

0    None
1    None
2    None
3    None
4    None
5    None
6    None
dtype: object

## other

In [9]:
"query examples"
{_.id:_.to_dict() for _ in races.where('Location','==','GT').stream()}

{'mychoice23feb': {'bibPattern': '\\d\\d\\d\\d',
  'Date': '2023-02-11',
  'Name': 'My Choice 2023 February',
  'Location': 'GT',
  'Waypoints': ['venue', 'end']}}

In [14]:
def copyRace(fromId,toId):
    fromRace=races.document(fromId).get().to_dict()
    print(fromRace)
    return races.document(toId).set(fromRace)

# copyRace('mychoice23apr','mychoice23mar')
# copyRace('mychoice23mar','mychoice23jan')

{'Date': DatetimeWithNanoseconds(2023, 3, 10, 23, 30, 0, 618000, tzinfo=datetime.timezone.utc), 'Waypoints': ['venue', 'start', 'end'], 'Name': 'My Choice 2023 March', 'Location': 'DC'}


update_time {
  seconds: 1679059217
  nanos: 514663000
}

In [18]:
def addNewRace(id):
    raceData = {'Date': "YYYY-MM-DD", 
                'Waypoints': ['venue', 'start', 'end'], 
                'Name': 'Default Race Year Month', 
                'Location': 'DC',
                'bibPatterm': r'\d\d\d\d'
               }
    update_time, city_ref = races.document(id).set(raceData)
    print(f'Added document with id {city_ref.id}')
    
addNewRace('mychoice23apr')

TypeError: cannot unpack non-iterable WriteResult object

# Misc

In [41]:
def getChildren(node):
    from collections.abc import Iterable

    if isinstance(node, Iterable):
        for c1 in node:
            print (c1._path,c1.document().get().to_dict())

            for d in c1.list_documents():
                getChildren(d)
 
    # single collection
    elif isinstance(node,firestore.CollectionReference): 
        
        for c1 in node.get():
            print (c1.__dict__,c1.to_dict())
            getChildren(c1)
 
        
    # end node
    else:
        try:
            print (node._path,
               # node.__dict__)
               node.get().to_dict())
        except:
            print ('error')
    
getChildren(runpix.fstore.collection('races'))

{'_reference': <google.cloud.firestore_v1.document.DocumentReference object at 0x0000020440C57A50>, '_data': {'Date': DatetimeWithNanoseconds(2023, 3, 10, 23, 30, 0, 618000, tzinfo=datetime.timezone.utc), 'Name': 'My Choice 2023 March', 'Location': 'DC', 'Waypoints': ['venue', 'start', 'end']}, '_exists': True, 'read_time': DatetimeWithNanoseconds(2023, 3, 14, 4, 34, 25, 527621, tzinfo=datetime.timezone.utc), 'create_time': DatetimeWithNanoseconds(2023, 3, 13, 7, 23, 6, 884067, tzinfo=datetime.timezone.utc), 'update_time': DatetimeWithNanoseconds(2023, 3, 13, 7, 24, 16, 876358, tzinfo=datetime.timezone.utc)} {'Date': DatetimeWithNanoseconds(2023, 3, 10, 23, 30, 0, 618000, tzinfo=datetime.timezone.utc), 'Name': 'My Choice 2023 March', 'Location': 'DC', 'Waypoints': ['venue', 'start', 'end']}
error
{'_reference': <google.cloud.firestore_v1.document.DocumentReference object at 0x0000020440F12C10>, '_data': {'Date': '2023-02-11', 'Name': 'My Choice 2023 February', 'Location': 'GT', 'Waypoi

In [16]:
for x in runpix.fstore.collection('races'):
    print(dir(x))
    print(x.document('test1').get().to_dict())

TypeError: 'CollectionReference' object is not iterable

# Images (Storage)

In [9]:
images = runpix.fstore.collection('images')
for doc in images.stream():
    print(f"{doc.id} => {doc.to_dict()['textAnnotations']}")

test^V4AFod9aCoRsEP62GnVm0gPYxXK2^2023-01-27T11:31:41.796Z.png => []
test^V4AFod9aCoRsEP62GnVm0gPYxXK2^test_20230115_090440[1].jpg => [{'topicality': 0, 'description': 'GARM\nO\nTATA\ntos\nTATA\nLATA\nMUMBAI\nMARAT\nTATA\nMARATHON\n42.195 KM\nF\n10641\nGYANA SAMAL\nIDFC FIRST Bank\n8369\nANUBHUTI CHATURVEDI\nIDFC FIRST Bank\nY\naquants\nCHANGE\nTATA\nB\nTATA\nG\nTATA\nMUMBAI\nMARATHON\n11148\nKRISHNA PATIL\nIDFC FIRST Bank\n42.195 KM\nB\n8\nTAT\nMUME\nMARA\nTATA CONSU\nTATA\nMARATHO', 'mid': '', 'locations': [], 'locale': 'und', 'boundingPoly': {'vertices': [{'y': 952, 'x': 302}, {'y': 952, 'x': 3683}, {'y': 2623, 'x': 3683}, {'y': 2623, 'x': 302}], 'normalizedVertices': []}, 'score': 0, 'properties': [], 'confidence': 0}, {'topicality': 0, 'description': 'GARM', 'mid': '', 'locations': [], 'locale': '', 'boundingPoly': {'vertices': [{'y': 2321, 'x': 302}, {'y': 2124, 'x': 776}, {'y': 2271, 'x': 837}, {'y': 2468, 'x': 363}], 'normalizedVertices': []}, 'score': 0, 'properties': [], 'con

# Storage

In [32]:
from firebase_admin import storage
bucket=firebase_admin.storage.bucket(name='run-pix.appspot.com', app=None) 

## list all blobs

In [202]:
# check existance
display(df_blob.name[3])
bucket.blob(df_blob.name[3]).exists()

'processed/mychoice23apr/2023-03-18T02:39:47.304Z~venue~avinashmane$gmail.com~capture.jpg'

True

In [203]:
blobs=[blob for blob in bucket.list_blobs()]
df_blob=pd.DataFrame([{"id":b.id,"name":b.name,"blob":b} for b in blobs])

In [206]:
# for i, b in enumerate(blobs):
#     print (i,b.name)
df_blob

,id,name,blob
0,run-pix.appspot.com/processed//1675677894996648,processed/,"<Blob: run-pix.appspot.com, processed/, 167567..."
1,run-pix.appspot.com/processed/default/2023-03-...,processed/default/2023-03-17T17:28:45.677Z~ven...,"<Blob: run-pix.appspot.com, processed/default/..."
2,run-pix.appspot.com/processed/default/2023-03-...,processed/default/2023-03-17T17:34:42.274Z~ven...,"<Blob: run-pix.appspot.com, processed/default/..."
3,run-pix.appspot.com/processed/default/_L3A2996...,processed/default/_L3A2996.jpg,"<Blob: run-pix.appspot.com, processed/default/..."
4,run-pix.appspot.com/processed/default/_L3A3012...,processed/default/_L3A3012.jpg,"<Blob: run-pix.appspot.com, processed/default/..."
...,...,...,...
372,run-pix.appspot.com/uploads/werun2023/_L3A3198...,uploads/werun2023/_L3A3198.jpg,"<Blob: run-pix.appspot.com, uploads/werun2023/..."
373,run-pix.appspot.com/uploads/werun2023/_L3A3199...,uploads/werun2023/_L3A3199.jpg,"<Blob: run-pix.appspot.com, uploads/werun2023/..."
374,run-pix.appspot.com/uploads/werun2023/_L3A3200...,uploads/werun2023/_L3A3200.jpg,"<Blob: run-pix.appspot.com, uploads/werun2023/..."
375,run-pix.appspot.com/uploads/werun2023/_L3A3201...,uploads/werun2023/_L3A3201.jpg,"<Blob: run-pix.appspot.com, uploads/werun2023/..."


In [39]:
"signed URL"
blobs[3].generate_signed_url(pd.Timestamp.today())

'https://storage.googleapis.com/run-pix.appspot.com/processed/mychoice23feb/2023-02-12T01%3A28%3A29.364Z%5Evenue%5Eavinashmane%24gmail.com%5E20230212_065828.jpg?Expires=1679098198&GoogleAccessId=firebase-adminsdk-spqa9%40run-pix.iam.gserviceaccount.com&Signature=MfFLAHv5J8xju%2BvZLS8uhW2CLJ7FXQU3eNNGOJHCxejfbcG2jBBuZuiwi%2BQVi5aYqVE70uYGMf4a7rXIaQNn4I5DaZuic%2BBwnstt0sftMLWTtTa2ZXFmHhog5jiyl18zQMS2oBrFE1fepTDLCzHktLWqD9lKvxK1tjffD1ERLkuAyRZMerhReLMBmedHMYLhB%2BoKLNo%2BBcoI9JHfkoIHoyal2M31c%2FxPoqJGSLemvE4TLMMePU%2FWK6ZB6VqEsb06%2FabVrGwbvjus2%2BOCoUK8cFOL6P1VCnRG%2FZ21GUvZJCuv9kRLeeuO22HSX%2BsNDwDP%2BC%2BlMfxCe%2FsGpC%2Fxd8NO4g%3D%3D'

In [239]:

from base64 import b64decode
base64_data = blocs[1].download_as_bytes() #"iVBORw0KGgoAAAANSUhEUgAABL ...  the rest of data "

ImportError: cannot import name 'Markdown' from 'IPython' (c:\sw\py310nb_env\Lib\site-packages\IPython\__init__.py)

In [55]:
base64_data[:100]

b"\xff\xd8\xff\xdb\x00C\x00\x06\x04\x05\x06\x05\x04\x06\x06\x05\x06\x07\x07\x06\x08\n\x10\n\n\t\t\n\x14\x0e\x0f\x0c\x10\x17\x14\x18\x18\x17\x14\x16\x16\x1a\x1d%\x1f\x1a\x1b#\x1c\x16\x16 , #&')*)\x19\x1f-0-(0%()(\xff\xdb\x00C\x01\x07\x07\x07\n\x08\n\x13\n\n\x13(\x1a\x16\x1a(((((((((("

In [56]:
display.Image(b64decode(base64_data))

In [90]:
# def newname(x):
#     arr=x.split('/')
#     arr[]
# gs://run-pix.appspot.com/thumbs/2023-02-12T01:25:41.084Z^venue^avinashmane$gmail.com^20230212_065538.jpg/2023-02-12T01:25:41.084Z^venue^avinashmane$gmail.com^20230212_065538.jpg
for i,b in enumerate(blobs):
    if 'thumbs/2023' in b.name :#and b.content_type=='image/jpeg':        
        newName="thumbs/mychoice23feb/"+b.name.split('/')[-1]
        print(i,b.name,newName)
        bucket.copy_blob(b,bucket,newName)

71 thumbs/2023-02-12T01:25:41.084Z^venue^avinashmane$gmail.com^20230212_065538.jpg/2023-02-12T01:25:41.084Z^venue^avinashmane$gmail.com^20230212_065538.jpg thumbs/mychoice23feb/2023-02-12T01:25:41.084Z^venue^avinashmane$gmail.com^20230212_065538.jpg


NotFound: 404 POST https://storage.googleapis.com/storage/v1/b/run-pix.appspot.com/o/thumbs%2F2023-02-12T01%3A25%3A41.084Z%5Evenue%5Eavinashmane%24gmail.com%5E20230212_065538.jpg%2F2023-02-12T01%3A25%3A41.084Z%5Evenue%5Eavinashmane%24gmail.com%5E20230212_065538.jpg/copyTo/b/run-pix.appspot.com/o/thumbs%2Fmychoice23feb%2F2023-02-12T01%3A25%3A41.084Z%5Evenue%5Eavinashmane%24gmail.com%5E20230212_065538.jpg?prettyPrint=false: No such object: run-pix.appspot.com/thumbs/2023-02-12T01:25:41.084Z^venue^avinashmane$gmail.com^20230212_065538.jpg/2023-02-12T01:25:41.084Z^venue^avinashmane$gmail.com^20230212_065538.jpg

In [70]:

".generate_signed_url() needs date"
new_id="run-pix.appspot.com/thumbs/mychoice23feb/2023-02-12T01:25:41.084Z^venue^avinashmane$gmail.com^20230212_065538.jpg/1678882880166980"
blobs[72].content_type

Signature:
blobs[72].rewrite(
    source,
    token=None,
    client=None,
    if_generation_match=None,
    if_generation_not_match=None,
    if_metageneration_match=None,
    if_metageneration_not_match=None,
    if_source_generation_match=None,
    if_source_generation_not_match=None,
    if_source_metageneration_match=None,
    if_source_metageneration_not_match=None,
    timeout=60,
    retry=<google.cloud.storage.retry.ConditionalRetryPolicy object at 0x0000026031EF2F50>,
)
Docstring:
Rewrite source blob into this one.

If :attr:`user_project` is set on the bucket, bills the API request
to that project.

:type source: :class:`Blob`
:param source: blob whose contents will be rewritten into this blob.

:type token: str
:param token:
    (Optional) Token returned from an earlier, not-completed call to
    rewrite the same source blob.  If passed, result will include
    updated status, total bytes written.

:type client: :class:`~google.cloud.storage.client.Client`
:param client:
  

In [71]:
bucket.copy_blob()

Signature:
gcs.copy_blob(
    blob,
    destination_bucket,
    new_name=None,
    client=None,
    preserve_acl=True,
    source_generation=None,
    if_generation_match=None,
    if_generation_not_match=None,
    if_metageneration_match=None,
    if_metageneration_not_match=None,
    if_source_generation_match=None,
    if_source_generation_not_match=None,
    if_source_metageneration_match=None,
    if_source_metageneration_not_match=None,
    timeout=60,
    retry=<google.cloud.storage.retry.ConditionalRetryPolicy object at 0x0000026031EF2F50>,
)
Docstring:
Copy the given blob to the given bucket, optionally with a new name.

If :attr:`user_project` is set, bills the API request to that project.

See [API reference docs](https://cloud.google.com/storage/docs/json_api/v1/objects/copy)
and a [code sample](https://cloud.google.com/storage/docs/samples/storage-copy-file#storage_copy_file-python).

:type blob: :class:`google.cloud.storage.blob.Blob`
:param blob: The blob to be copied.



# Bulk Bib Upload

Data copied from clipboard from Excel:

Columns copied are (case sensitive)
* Bib
* Name
* Status
* Race

In [85]:
raceId = 'werun2023'
bibs=runpix.fstore.collection(f'races/{raceId}/bibs')

In [72]:
"Check Race details"
def dictUtf(mydict): 
    return {k: str(v).encode("utf-8") for k,v in mydict.items()}

races.document(raceId).get().to_dict()

{'bibPattern': '\\d{4}',
 'Date': '2023-03-05',
 'Location': 'Nigadi Pradhikaram',
 'Name': 'We Run 2023 (4th Edition)',
 'Waypoints': ['venue', 'route']}

##  Copy the data
Data copied from clipboard from Excel:

Columns copied are (case sensitive)
* Bib
* Name
* Status
* Race

In [60]:
df_startlist=pd.read_clipboard()
display("columns in clipboard:",df_startlist.columns)
_required_columns='Bib	Name	Status	Race'.split('\t')
_found_columns = [_x for _x in _required_columns if  _x in df_startlist.columns]
display("columns found ",_found_columns)
if len(_found_columns)<len(_required_columns): print("PLEASE WAIT "+('X'*10+' ')*3)


'columns in clipboard:'

Index(['Bib', 'Name', 'Attendee Email', 'Status', 'Race'], dtype='object')

'columns found '

['Bib', 'Name', 'Status', 'Race']

In [62]:
df_startlist=df_startlist[_required_columns]

In [98]:
"code"
bibtoDict=lambda x: x.to_dict()
bibtoDict(df_startlist.loc[0,:])  # test

def saveBibtoRace(bibData,bibsCollection):
    _bibData=bibtoDict(bibData)
    _bibData['Bib']=str(_bibData['Bib'])
    bibsCollection.document(_bibData['Bib']
                           ).set(_bibData)
saveBibtoRace(df_startlist.loc[0,:],bibsCollection=bibs,)  # test    
# df_startlist.apply(, axis=1)


In [99]:
" upload all bibs"
# df_startlist.apply(bibtoDict, axis=1)  #test
df_startlist.apply(lambda x: saveBibtoRace(x,bibs), axis=1) 

0       None
1       None
2       None
3       None
4       None
        ... 
1030    None
1031    None
1032    None
1033    None
1034    None
Length: 1035, dtype: object

In [100]:
bibs.count()

In [241]:
len(list( bibs.list_documents()))
   

1035

# Update images to bibs 

Add list of images to the bibs

# Bulk Uploadlist_documents

In [219]:
import yaml
import os
cfg={}
cfg.update(yaml.safe_load("""
uploads:
    werun2023:
        path: D:\We Run 2023\JPEG
        #path: D:\We Run 2023\Vaibhav\JPEG
        fileRange: ["","XX"]
        waypoint: venue
        userid: vaibhav
"""))
cfg

{'uploads': {'werun2023': {'path': 'D:\\We Run 2023\\JPEG',
   'fileRange': ['', 'XX'],
   'waypoint': 'venue',
   'userid': 'vaibhav'}}}

In [220]:
"check reading file"
from IPython.display import Image
# Image(filename=r"D:\We Run 2023\Vaibhav\JPEG\S_G01782.jpg") 

In [242]:
# !gsutil

In [222]:
df_blob.head(2)

,id,name,blob
0,run-pix.appspot.com/processed//1675677894996648,processed/,"<Blob: run-pix.appspot.com, processed/, 167567..."
1,run-pix.appspot.com/processed/default/2023-03-...,processed/default/2023-03-17T17:28:45.677Z~ven...,"<Blob: run-pix.appspot.com, processed/default/..."


In [251]:
"""
    find object from gcs
    
"""
display(Markdown("## rename blob"))
# bucket.blob('uploads/werun2023/_L3A3192.jpg').rewrite
# blob.rewrite?
bname='2023-03-13T19:23:14.739819~general~vaibhav~_L3A2997.jpg'
def renameBlob(bname):
    bucket.rename_blob(bucket.blob(bname),folder+bname)
# renameBlob(bname) 
"use 2"
bucket.blob('uploads/werun2023/2023-03-13T19:23:11.098916~general~vaibhav~_L3A2996.jpg').time_created

## rename blob

TypeError: 'NoneType' object is not callable

In [247]:
"""
Manually rename to retrigger the processing...
- You can change waypoint from venue to general
"""
"* Storage: /uploads/race/time~wpt~user~loc~file    # uploaded images"
import os
from datetime import datetime
i=0
user='vaibhav'
place='general'
folder='uploads/werun2023/'
def getFileName(file):
    return file.split("\\")[-1]
def getFileIsoDate(file):
    return datetime.fromtimestamp( os.path.getctime(file) ).isoformat()
def getNewfileName(file,user=user,place=place):
    return "~".join([getFileIsoDate(file),place,user,getFileName(file)])

for lot in cfg['uploads'].keys():
    print("raceId",lot)
    for root, dirs, files in os.walk(cfg['uploads'][lot]['path'], topdown=False):
       for name in files:
            i=i+1
            path=os.path.join(root, name)
            new=folder+getNewfileName(path)
        
            # blob=bucket.blob(folder+name) #used when renaming to diff filenames
            blob=bucket.blob(folder+new)
            
            if blob.exists():
                bucket.rename_blob(blob,new)
                print(i,name,new)
            else:
                blob.upload_from_filename(path)
                print(i,f'uploading {path} to {new}')
                
            # if (i>20): raise  #stopper
       for name in dirs:
          print(">>>>>>",os.path.join(root, name))

raceId werun2023
1 uploading D:\We Run 2023\JPEG\_L3A2996.jpg to uploads/werun2023/2023-03-13T19:23:11.098916~general~vaibhav~_L3A2996.jpg
2 uploading D:\We Run 2023\JPEG\_L3A2997.jpg to uploads/werun2023/2023-03-13T19:23:14.739819~general~vaibhav~_L3A2997.jpg
3 uploading D:\We Run 2023\JPEG\_L3A2998.jpg to uploads/werun2023/2023-03-13T19:23:17.802551~general~vaibhav~_L3A2998.jpg
4 uploading D:\We Run 2023\JPEG\_L3A2999.jpg to uploads/werun2023/2023-03-13T19:23:21.443454~general~vaibhav~_L3A2999.jpg
5 uploading D:\We Run 2023\JPEG\_L3A3000.jpg to uploads/werun2023/2023-03-13T19:23:24.896841~general~vaibhav~_L3A3000.jpg
6 uploading D:\We Run 2023\JPEG\_L3A3002.jpg to uploads/werun2023/2023-03-13T19:23:28.772136~general~vaibhav~_L3A3002.jpg
7 uploading D:\We Run 2023\JPEG\_L3A3003.jpg to uploads/werun2023/2023-03-13T19:23:31.631728~general~vaibhav~_L3A3003.jpg


KeyboardInterrupt: 

In [216]:
df_blob.query("name.str.contains('G017')")

,id,name,blob


# Other